# Export Report

Generate a token following the directions [here](https://ibm.github.io/Environmental-Intelligence-Suite/api-tokens.html) and paste the token into the quotes for `AUTH_TOKEN` below.

_____________________

In [155]:
AUTH_TOKEN = ''
API_ENDPOINT = 'foundation.agtech.ibm.com/v2'

In [156]:
import os
import json
import pprint
import requests
import geopandas as gpd

In [157]:
os.environ["API_ENDPOINT"] = API_ENDPOINT
os.environ["AUTH_TOKEN"]= AUTH_TOKEN

ASSET_MGMT = {
    'host': os.environ.get('API_ENDPOINT', 'Run the notebook to set env vars')
}

HEADERS = {
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + os.environ.get("AUTH_TOKEN", 'Run the notebook to set env vars'),
    'Content-Type': 'application/json; charset=UTF-8'
}

PRINTER = pprint.PrettyPrinter(indent = 2)

In [158]:
def getHierarchy():
    get_url = 'https://{}/hierarchy'.format(ASSET_MGMT['host'])
    
    r = requests.get(url=get_url, headers=HEADERS)
    return json.loads(r.text) if (r.status_code == 200) else r.raise_for_status()


def getAsset(input_type=None, parentID=None, limit=20, start=0, idsOnly=False, includeScalarInfo=False, includeAssetGeometry=False):
    
    get_url = 'https://{}/asset'.format(ASSET_MGMT['host'])
    
    params = {
        'limit': limit,
        'start': start,
        'idsOnly': idsOnly,
        'includeAssetGeometry': includeAssetGeometry,
        'projection': 4326
    }
    
    if input_type is not None:
        params['inputType'] = input_type
        
    if parentID is not None:
        params['parentReferenceUuid'] = parentID

    if includeScalarInfo is True:
        params['includeScalarInfo'] = includeScalarInfo
    
    r = requests.get(url=get_url, headers=HEADERS,  params=params)
    
    return json.loads(r.text) if (r.status_code == 200) else r.raise_for_status()


def getScalarType(scalar_type=None):
    get_url = 'https://{}/scalar/type'.format(ASSET_MGMT['host'])
    
    params = {}
    
    if scalar_type is not None:
        params['type'] = scalar_type
    
    r = requests.get(url=get_url, headers=HEADERS,  params=params) 
    return json.loads(r.text) if (r.status_code == 200) else r.raise_for_status()


def postReportJob(job_name, job_params_list, inputType, territory_uuid=None, scalar_data=True, wkt_geom=False, limit=None):
    post_url = 'https://{}/report/job'.format(ASSET_MGMT['host'])
    
    data = {
        'jobName': job_name,
        'jobType': 'ASSET_SEARCH',
        'jobRequest': {
            'inputType': inputType,
            'nameMatch': 'STARTS_WITH',
            'includeScalarInfo':  scalar_data,
            'projection': 4326,
            'uuidsOnly': False,
            'start': 0,
            'limit': limit,
            'parentUuid': territory_uuid,
            'includeAllChildren': True
        },
        'jobParams': job_params_list
    }
    
    if wkt_geom and 'geometry' not in data['jobParams']:
        data['jobParams'].append('geometry')
    
    PRINTER.pprint(data)
    r = requests.post(url = post_url, data = json.dumps(data), headers = HEADERS) 
    return json.loads(r.text)


def getReportStatus(jobid, includeStage=True):
    
    get_url = 'https://{}/report/{}/status?includeStage={}'.format(ASSET_MGMT['host'],jobid,includeStage)
    r = requests.get(url = get_url, headers = HEADERS) 
    return json.loads(r.text) if (r.status_code == 200) else r.raise_for_status()

    
def fetchReport(jobid, filepath, fileformat):

    get_url = 'https://{}/report/{}/data?format={}'.format(ASSET_MGMT['host'], jobid, fileformat)
    r = requests.get(url=get_url, headers=HEADERS)
    
    if fileformat == 'geojson':
        with open(filepath + '.geojson', 'w') as outfile:
            outfile.write(r.text)
            
    elif fileformat == 'csv':
        with open(filepath + '.csv', 'w') as outfile:
            outfile.write(r.text)

___

_____________________

### Generate Report For All Territories

**(1) Configure The Request**

First, we will define a name for the report

In [159]:
job_name = 'Demo'

Next, make an API call get to get all available scalar types

In [160]:
scalar_types = getScalarType()
PRINTER.pprint(scalar_types)

{ 'scalarTypes': [ { 'description': 'Total area of asset',
                     'displayFormat': 'area',
                     'displayName': 'Area',
                     'id': 41,
                     'propertyName': 'area_cor',
                     'type': 'Kpi',
                     'unit': 'meters^2',
                     'uuid': 'c38b0f8d-7439-40da-a544-c26b6b7d5a73'},
                   { 'description': 'Length of asset',
                     'displayFormat': 'meters',
                     'displayName': 'Length of asset',
                     'id': 42,
                     'propertyName': 'length_cor',
                     'type': 'Kpi',
                     'unit': 'meters',
                     'uuid': 'ca42c431-9535-4acc-bb78-cbaaac073ed3'},
                   { 'description': 'Count of veg polygons within 10ft',
                     'displayFormat': 'num',
                     'displayName': 'Count of veg polygons within 10ft',
                     'id': 43,
                 

Create a list of all scalars from the API call above.

In [161]:
all_scalars = []
for scalar in scalar_types['scalarTypes']:
    all_scalars.append(scalar['propertyName'])

Then, fetch all custom properies from the hierarchy

In [162]:
hierarchy = getHierarchy()
custom_props = []
for level in hierarchy['hierarchyList'][0]['hierarchyLevels']:
    for filter in level['displayFilters']:
        if filter['type'] == 'CustomProperty':
            if filter['key'] not in custom_props and filter['key'] != 'length':
                custom_props.append(filter['key'])

Finally, assign the `job_params` to `all_scalars + custom_props` and make the API call below to create a report.

In [163]:
job_params = all_scalars + custom_props
createReport = postReportJob(job_name, job_params, 'SEGMENT', wkt_geom=True)
PRINTER.pprint(createReport)

{ 'jobName': 'Demo',
  'jobParams': [ 'area_cor',
                 'length_cor',
                 'dist_less_than_3_048m',
                 'dist_less_than_6_096m',
                 'total',
                 'min_3d_distance',
                 'avg_vert_dist',
                 'min_2d_distance',
                 'less_than_percent_3_048m',
                 'less_than_percent_6_096m',
                 'area_trees_over_0m',
                 'area_trees_over_9_144m',
                 'percent_area_trees_over_9_144m',
                 'area_trees_over_18_288m',
                 'percent_area_trees_over_18_288m',
                 'trees_over_wire',
                 'trees_over_9_144m',
                 'trees_over_18_288m',
                 'wire_height',
                 'length_with_veg',
                 'tree_height_avg',
                 'tree_coverage_percentage',
                 'tree_height_max',
                 'fallin_tree_count',
                 'action_recommended',
         

___

**(2) Next, get the status of the job report**

Change `includeStage` to `False` to remove the detailed stages output

In [166]:
jobid = createReport['jobId']
reportStatus = getReportStatus(jobid, includeStage=True)
PRINTER.pprint(reportStatus)
print("\nJob Status:", reportStatus["status"])

{ 'createdDate': '2022-02-24T18:38:30Z',
  'jobId': '5aefd09b-6db3-476c-83da-6a57065a1c95',
  'jobName': 'Demo',
  'lastModifiedDate': '2022-02-24T18:38:41Z',
  'organisation': 'wocvegmdemo',
  'stages': [ { 'description': 'upload [/tmp/tmp44_5hnyi_geojson.zip] to COS',
                'stage': 'upload',
                'status': 'COMPLETE',
                'timestamp': '2022-02-24T18:38:41Z'},
              { 'description': 'upload [/tmp/tmp44_5hnyi_geojson.zip] to COS',
                'stage': 'upload',
                'status': 'START',
                'timestamp': '2022-02-24T18:38:40Z'},
              { 'description': 'Build zip file for geojson',
                'stage': 'Build ZIP file',
                'status': 'COMPLETE',
                'timestamp': '2022-02-24T18:38:40Z'},
              { 'description': 'Build zip file for geojson',
                'stage': 'Build ZIP file',
                'status': 'START',
                'timestamp': '2022-02-24T18:38:40Z'},
          

___

**(3) If job status above is "READY", *specify the filepath in quotes below* where you would like the save the report, then run this code to download the report. See commented out first line for example.**

`fileformat=geojson` can be changed to `fileformat=csv` in order to download the file as a CSV instead.

In [167]:
filepath = '../../../../Desktop/Example_Report'

In [168]:
fetchReport(jobid, filepath, fileformat='geojson')

___

______________

### Generate Report at the Territory Level
**(1) First, get all territories**

In [93]:
territories = getAsset(limit=5, input_type='TERRITORY', includeAssetGeometry=False)
PRINTER.pprint(territories)

{ 'features': [ { 'projection': 4326,
                  'properties': { 'Territory': 'NAPERVILLE_NORTH',
                                  'UNIQUE_ROW_ID_COLS': 'Territory',
                                  'area': 7642082.199489633,
                                  'box': { 'east': -88.126626,
                                           'north': 41.801992,
                                           'south': 41.779182,
                                           'west': -88.172316},
                                  'centroid': { 'latitude': 41.78991330518308,
                                                'longitude': -88.15144919564224},
                                  'deleted': False,
                                  'geometryBufferSize': 0.0,
                                  'hierarchyLevelType': 'TERRITORY',
                                  'ianaTimeZone': 'America/Chicago',
                                  'inputType': 'ASSET',
                                  'name': 'N

Save the `TERRITORY` UUIDs to `territory_uuids`

In [94]:
territory_uuids = []
for territory in territories['features']:
    territory_uuids.append(territory['uuid'])
print('\nTerritory UUIDS:\n')
PRINTER.pprint(territory_uuids)


Territory UUIDS:

['fa601936-4ce8-4054-8e17-84b5746febd3', 'c33c5a6b-439a-4b91-96fd-d6f537ccc733']


___

**(2) Next, generate the report**

Use the `job_params` configured above and the first ID from `territory_uuids` to generate a report just for the first territory, `NAPERVILLE_NORTH`

In [95]:
createReport_territory = postReportJob(job_name, job_params, 'SEGMENT', wkt_geom=True, territory_uuid=territory_uuids[0])
PRINTER.pprint(createReport_territory)

{ 'jobName': 'Demo',
  'jobParams': [ 'area_cor',
                 'length_cor',
                 'dist_less_than_3_048m',
                 'dist_less_than_6_096m',
                 'total',
                 'min_3d_distance',
                 'avg_vert_dist',
                 'min_2d_distance',
                 'less_than_percent_3_048m',
                 'less_than_percent_6_096m',
                 'area_trees_over_0m',
                 'area_trees_over_9_144m',
                 'percent_area_trees_over_9_144m',
                 'area_trees_over_18_288m',
                 'percent_area_trees_over_18_288m',
                 'trees_over_wire',
                 'trees_over_9_144m',
                 'trees_over_18_288m',
                 'wire_height',
                 'length_with_veg',
                 'tree_height_avg',
                 'tree_coverage_percentage',
                 'tree_height_max',
                 'fallin_tree_count',
                 'action_recommended',
         

___

**(3) Next, get the status of the job report**

Change `includeStage` to `False` to remove the detailed stages output

In [97]:
jobid_territory = createReport_territory['jobId']
reportStatus = getReportStatus(jobid_territory, includeStage=True)
PRINTER.pprint(reportStatus)
print("\nJob Status:", reportStatus["status"])

{ 'createdDate': '2022-02-24T17:30:29Z',
  'jobId': '0e882084-7556-4a96-8000-5031d0e6bd9c',
  'jobName': 'Demo',
  'lastModifiedDate': '2022-02-24T17:30:39Z',
  'organisation': 'wocvegmdemo',
  'stages': [ { 'description': 'upload [/tmp/tmp_5737uec_geojson.zip] to COS',
                'stage': 'upload',
                'status': 'COMPLETE',
                'timestamp': '2022-02-24T17:30:38Z'},
              { 'description': 'upload [/tmp/tmp_5737uec_geojson.zip] to COS',
                'stage': 'upload',
                'status': 'START',
                'timestamp': '2022-02-24T17:30:38Z'},
              { 'description': 'Build zip file for geojson',
                'stage': 'Build ZIP file',
                'status': 'COMPLETE',
                'timestamp': '2022-02-24T17:30:38Z'},
              { 'description': 'Build zip file for geojson',
                'stage': 'Build ZIP file',
                'status': 'START',
                'timestamp': '2022-02-24T17:30:37Z'},
          

___

**(4) If job status above is "READY", specify the filepath in quotes below where you would like the save the report, then run this code to download the report. See commented out first line for example.**

`fileformat=geojson` can be changed to `fileformat=csv` in order to download the file as a CSV instead.

In [98]:
filepath = '../../../../Desktop/Example_Territory_Report'

In [100]:
fetchReport(jobid_territory, filepath, fileformat='geojson')

___

___

___